In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from io import StringIO

In [49]:
def contains_pattern(row, pattern):
      return any(pattern in str(cell) for cell in row)
def remove_after_colon(s):
      return s.split(':')[0]

def process_df(filename):
  #read and clean
  with open(filename) as f:
    lines = [line for line in f if not line.startswith('##')]

  # Parse the filtered content with pandas
  q = pd.read_csv(StringIO(''.join(lines)),sep="\t")

  #q=pd.read_csv(filename,comment='##',sep="\t")
  q.drop(columns=["#CHROM","ID","QUAL","FILTER","INFO","FORMAT"],inplace=True)
  col_names=[]
  for c in q.columns:
      if c[0]=="/":
        n=c.split("/")[7]
        s=n.split("_")[1]
        col_names.append(s)
      else:
        col_names.append(c)
  q.columns=col_names

  #only SNP and biallelic
  df_filtered = q[q['REF'].str.len() == 1]
  df_filtered = df_filtered[df_filtered['ALT'].str.len() == 1]

  #take out quality info, leave just genotype
  df_filtered['POS'] = df_filtered['POS'].astype(str)
  df_filtered = df_filtered.applymap(remove_after_colon)

  #remove heterozygotes
  pattern = "0/1"
  df_filtered = df_filtered[~df_filtered.apply(lambda row: contains_pattern(row, pattern), axis=1)]

  # Replace "0" with values from "REF" column
  for col in df_filtered.columns[3:]:
    df_filtered[col] = df_filtered.apply(lambda row: row['REF'] if row[col] == '0/0' else row[col], axis=1)
    df_filtered[col] = df_filtered.apply(lambda row: row['ALT'] if row[col] == '1/1' else row[col], axis=1)

  #count precision of marker
  bases=[]
  freq = []
  for index, row in df_filtered.iterrows():
     most_c = Counter(row[3:]).most_common(1)[0] #('G', 15)
     bases.append(most_c[0])
     freq.append(most_c[1]/len(row[3:]))

  df_filtered=df_filtered.iloc[:,:3]
  df_filtered["val"]=bases
  df_filtered["percentage"]=freq

  #keep rows where everyone is the same
  #q=df_filtered[df_filtered[df_filtered.columns[3:]].nunique(axis=1) == 1]

  return df_filtered

In [78]:
mypath='/gpfs1/home/c/p/cpetak/WGS/local_pca_pipe/'
mychr='NW_022145610.1'
posi='_30779143_31460853_'#'_29427741_29776196_'#'_16610900_16737625_'#'_8291260_9094120_' #'_28950395_29566247_' #'_33842803_34582517_' #'_14219351_14298524_' #'_12702886_16793794_' # #

q=process_df(mypath+mychr+posi+'homoq.vcf')
#q = q.rename(columns={q.columns[-1]: 'q'})
print(q) #most common base and it's frequency

p=process_df(mypath+mychr+posi+'homop.vcf')
#p = p.rename(columns={p.columns[-1]: 'p'})
print(p) #most common base and it's frequency



/tmp/ipykernel_49864/992128992.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df_filtered.applymap(remove_after_colon)


             POS REF ALT  val  percentage
0           2808   A   T    A         1.0
1           2928   G   A    G         0.8
3           3139   A   T    A         0.6
4           3152   T   C    T         0.6
5           3186   T   G  ./.         0.4
...          ...  ..  ..  ...         ...
409511  35903081   A   G    G         1.0
409513  35903829   A   T    A         1.0
409514  35903845   T   A    T         1.0
409517  35904273   G   A    G         1.0
409518  35907363   C   A    A         1.0

[135933 rows x 5 columns]


/tmp/ipykernel_49864/992128992.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df_filtered.applymap(remove_after_colon)


             POS REF ALT val  percentage
92         11636   G   T   T    1.000000
116        25517   G   A   A    1.000000
119        25599   C   A   A    1.000000
631        77717   G   A   A    1.000000
1448      114313   T   C   C    1.000000
...          ...  ..  ..  ..         ...
409258  35859748   T   A   A    1.000000
409319  35865883   G   T   T    0.986667
409355  35873280   G   A   A    0.853333
409472  35891427   A   G   G    0.973333
409493  35897869   G   A   A    0.946667

[7876 rows x 5 columns]


In [79]:
#out of all the positions on this chromosome, these are consistenlty different between the orientations.

merged_df = pd.merge(q, p, on='POS', how='inner') #only keep positions that are present in both p and q
nonuniqe=merged_df[merged_df[merged_df.columns[[3,7]]].nunique(axis=1) != 1] #only keep positions that are different between p and q
df = nonuniqe[~nonuniqe.apply(lambda row: contains_pattern(row, "/"), axis=1)]
df = df.drop(['REF_y', 'ALT_y'], axis=1)
df = df.rename(columns={'REF_x': 'REF','ALT_x': 'ALT', 'val_x': 'q', 'val_y': 'p','percentage_x': 'Frequency among q','percentage_y': 'Frequency among p' })

print(df)

df = df[df["Frequency among q"]>0.9]
df = df[df["Frequency among p"]>0.9]

df.to_csv(f'{mypath+mychr+posi}snp_markers.csv', index=False)

Empty DataFrame
Columns: [POS, REF, ALT, q, Frequency among q, p, Frequency among p]
Index: []


In [80]:
len(df)

0

In [81]:
posi

'_30779143_31460853_'

In [87]:
import pandas as pd
import glob
import os

# Find all CSV files that contain 'snp_marker' in the filename
csv_files = glob.glob("*snp_marker*.csv")

for file in csv_files:
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Round all numeric columns to 3 decimal places
    df = df.round(3)
    
    # Save it back (overwrite original)
    df.to_csv(file, index=False)

    print(f"Processed: {file}")


Processed: NW_022145600.1_33842803_34582517_snp_markers.csv
Processed: NW_022145597.1_14219351_14298524_snp_markers.csv
Processed: NW_022145594.1_39429440_42445994_snp_markers.csv
Processed: NW_022145594.1_12702886_16793794_snp_markers.csv
Processed: NW_022145610.1_30779143_31460853_snp_markers.csv
Processed: NW_022145606.1_16610900_16737625_snp_markers.csv
Processed: NW_022145601.1_28950395_29566247_snp_markers.csv
Processed: NW_022145609.1_29427741_29776196_snp_markers.csv
Processed: NW_022145603.1_8291260_9094120_snp_markers.csv
